<a href="https://colab.research.google.com/github/sanaelk/Springboard/blob/main/Capstone_Two_Data_Wrangling_(Updated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


Data source. I use Women’s International Football Results on Kaggle, specifically the goalscorers.csv file that logs one row per goal event in women’s international matches.  I work from goalscorers.csv to practice event→match aggregation.
Kaggle direct link ( https://www.kaggle.com/datasets/martj42/womens-international-football-results)

Objectives

  Develop a model that uses past match results to forecast the outcomes of upcoming women’s international football games. Convert goal-level data into match level outcomes (WIN/DRAW/LOSS) for building a football match prediction model.

In [ ]:
from google.colab import files



uploaded = files.upload()

Saving goalscorers.csv to goalscorers.csv


In [ ]:
#Load the data
print("1. LOADING AND INITIAL EXPLORATION")
print("-" * 50)
df = pd.read_csv('goalscorers.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")





In [ ]:
# Initial data inspection
print("\nInitial data overview:")
df.info()
print(f"\nFirst few records:")
print(df.head())


Need to convert the whole dataset from goal-driven data to match-level data to predict outcomes (Win/Draw/Loss). At the moment each row is whatever goal scored not who won or lost or drew.

In [ ]:
# changing 'date' to 'datetime' just to make it clear it's an actual timestamp I can use for sorting and grouping
#using 'date' as a timestamp can be confusing when trying to recall information
# 'date' comes in as plain text in the DataFrame, so renaming to 'datetime' makes it clear I'm turning it into a real datetime object

df = df.rename(columns={'date': 'datetime'})

# Drop rows with missing datetime (cannot aggregate into matches otherwise).
missing_dates = df['datetime'].isnull().sum()
print(f"Rows with missing datetime: {missing_dates}")
df = df.dropna(subset=['datetime'])

# Convert to proper datetime format and check for invalid dates
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
invalid_dates = df['datetime'].isnull().sum()
if invalid_dates > 0:
    print(f"Rows with invalid dates: {invalid_dates}")
    df = df.dropna(subset=['datetime'])



In [ ]:
# Check & remove exact duplicate goal-event rows (prevents double-counted goals)
candidate_cols = ['datetime', 'home_team', 'away_team', 'team', 'scorer', 'minute', 'own_goal', 'penalty']
dup_count = df.duplicated(subset=candidate_cols, keep=False).sum()
print(f"Exact duplicate goal-event rows: {dup_count}")
if dup_count > 0:
    df = df.drop_duplicates(subset=candidate_cols, keep='first')

#check consistency of names

home_teams = df['home_team'].unique()
away_teams = df['away_team'].unique()
all_teams = set(home_teams).union(set(away_teams))

print(f"Unique home teams: {len(home_teams)}")
print(f"Unique away teams: {len(away_teams)}")
print(f"Total unique teams: {len(all_teams)}")
print("Sample team names:", sorted(list(all_teams))[:10])

print("\n3. AGGREGATING TO MATCH-LEVEL DATA")
print("-" * 50)



{'Poland', 'Zimbabwe', 'Hungary', 'Argentina', 'Sweden', 'Costa Rica', 'North Macedonia', 'Romania', 'Mexico', 'Haiti', 'Albania', 'Panama', 'Great Britain', 'Slovenia', 'Andorra', 'Canada', 'Russia', 'Latvia', 'Kazakhstan', 'Italy', 'Ghana', 'Malta', 'England', 'Bosnia and Herzegovina', 'Vietnam', 'Netherlands', 'Morocco', 'Colombia', 'Azerbaijan', 'Thailand', 'Serbia', 'New Zealand', 'Armenia', 'Switzerland', 'Northern Ireland', 'Faroe Islands', 'Australia', 'Cameroon', 'Montenegro', 'Greece', 'Brazil', 'France', 'Scotland', 'Denmark', 'Austria', 'Ukraine', 'Georgia', 'Moldova', 'Japan', 'Belgium', 'Slovakia', 'Ivory Coast', 'China PR', 'Kosovo', 'Finland', 'Nigeria', 'Luxembourg', 'Germany', 'South Korea', 'Turkey', 'United States', 'Czech Republic', 'Iceland', 'Cyprus', 'Israel', 'Bulgaria', 'Estonia', 'Wales', 'Jamaica', 'North Korea', 'Taiwan', 'Croatia', 'Zambia', 'Republic of Ireland', 'Lithuania', 'Portugal', 'Ecuador', 'Norway', 'Chile', 'Belarus', 'Equatorial Guinea', 'South

In [ ]:
# Count goals and create match outcomes - FIXED: Use include_groups=False to avoid deprecation warning
matches = (
    df.groupby(['datetime', 'home_team', 'away_team'])
    .apply(lambda x: pd.Series({
        'home_goals': (x['team'] == x['home_team']).sum(),
        'away_goals': (x['team'] == x['away_team']).sum(),
        'total_goals': len(x),
        'penalty_goals': x['penalty'].sum(),
        'own_goals': x['own_goal'].sum()
    }), include_groups=False)
    .reset_index()
)

# Determine results
def match_result(row):
    if row['home_goals'] > row['away_goals']:
        return 'Home Win'
    elif row['home_goals'] < row['away_goals']:
        return 'Away Win'
    else:
        return 'Draw'



/tmp/ipython-input-823283507.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [ ]:
#Apply the match_result function
matches['result'] = matches.apply(match_result, axis=1)

print("\n4. DATA QUALITY CHECKS")
print("-" * 50)

# Check for extreme scores (outliers)
matches['goal_difference'] = abs(matches['home_goals'] - matches['away_goals'])
extreme_matches = matches[matches['goal_difference'] >= 10]
print(f"Matches with goal difference >= 10: {len(extreme_matches)}")



,datetime,home_team,away_team,home_goals,away_goals
0,1984-04-08,England,Denmark,2,1
1,1984-04-08,Italy,Sweden,2,3
2,1984-04-28,Denmark,England,0,1
3,1984-04-28,Sweden,Italy,2,1
4,1984-05-12,Sweden,England,1,0


In [ ]:
# Check for duplicate matches
duplicate_matches = matches.duplicated(subset=['datetime', 'home_team', 'away_team']).sum()
print(f"Duplicate matches found: {duplicate_matches}")

print("\n5. FINAL DATASET SUMMARY")
print("-" * 50)

print("Match results distribution:")
result_counts = matches['result'].value_counts()
print(result_counts)

print(f"\nFinal matches dataset shape: {matches.shape}")
print(f"Date range: {matches['datetime'].min()} to {matches['datetime'].max()}")
print(f"Average home goals: {matches['home_goals'].mean():.2f}")
print(f"Average away goals: {matches['away_goals'].mean():.2f}")

# Save the processed data
matches.to_csv('matches.csv', index=False)
print("\nSaved as 'matches.csv'")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>